In [ ]:
import sys
sys.path.append("..")

import torch
from src.model import ASRModel
from src.dataset import idx2char

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = ASRModel(len(idx2char)).to(DEVICE)
model.load_state_dict(torch.load("../best_model_gru256.pt", map_location=DEVICE))
model.eval()

print("Model loaded successfully")

Model loaded successfully


C:\Users\Asus\AppData\Local\Temp\ipykernel_34944\3735552336.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../best_model_512.pt", map

In [10]:
from torch.utils.data import DataLoader
from src.dataset import ASRDataset, collate_fn
from src.beam_decoder import ctc_beam_search

dataset = ASRDataset("../data/librispeech/dev_clean_manifest.csv")
loader = DataLoader(dataset, batch_size=1, collate_fn=collate_fn)

features, transcripts, _, _ = next(iter(loader))
features = features.to(DEVICE)

with torch.no_grad():
    logits = model(features)
    log_probs = logits.log_softmax(dim=-1)

pred_text = ctc_beam_search(log_probs[0].cpu(), idx2char, beam_width=5)
true_text = "".join([idx2char[i.item()] for i in transcripts[0] if i.item() != 0])

print("PRED :", pred_text)
print("TRUE :", true_text)


PRED : MSTE CFOTERSEA POLSO THO MAO COASIS E WER GLAT WOTOMHESCOSPO
TRUE : MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL


In [11]:
from jiwer import wer, cer

print("CER:", cer(true_text, pred_text))
print("WER:", wer(true_text, pred_text))


CER: 0.5168539325842697
WER: 1.0
